### NLTK Path

The code cell below is used for setting the NLTK path on DCS machines and Joshua. On my personal computer where this assignment was done, I had downloaded the NLTK corpus using **nltk.download()**


In [22]:
import getpass
username = getpass.getuser()

if username is not "ArghaWin10" or username is not "arghasarkar":
    import nltk
    nltk.data.path.append('/modules/cs918/nltk_data/')


# Sanity check

Sanity check to ensure that the file exists before proceeding further

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import testsets
import evaluation

from os.path import normpath
import os

# TODO: load training data
FILE_NAME = "./twitter-dev-data.txt"

if os.path.isfile(FILE_NAME):
    print("The file has been found.")
else:
    raise IOError("File not found.")

The file has been found.


# Sample object

This sample object will contain the tweet_id, sentiment and the tweet content. 

In [35]:

class Tweet:
    
    def __init__(self, raw_str):
        
        self.raw_str = raw_str
        self.tweet_id, self.sentiment, self.tweet = self.parse_raw_tweet()
        
    
    def parse_raw_tweet(self):
        parts = self.raw_str.split("\t")
        return parts[0], parts[1], parts[2]
    
    
    def __str__(self):
        return ("Tweet ID: {} Sentiment: {} Tweet: {}".format(str(self.tweet_id), self.sentiment, self.tweet))
    

''' 
Test code below. IGNORE -------
'''
# rt1 = "735752723159607191	positive	Shay with Bentley and Bella, in their sunday best :) http://t.co/SUMZBSTrkW"
# rt2 = "529243425878060644	negative	Dear MSM, CNN bitches, every election one turning point..there you go again I will not use my opponents youth & NOW \"basket of deplorables\""

# t1 = Tweet(rt1)
# t2 = Tweet(rt2)

# print(t1)
# print(t2)

' \nTest code below. IGNORE -------\n'

# Corpus 

Corpus object to read the data and preprocess it

In [21]:
import json
import itertools

from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize

class Corpus:
    
    def __init__(self, file_path):
        # Private vars
        self._corpus_loaded = False
#         self._corpus_parsed = False
#         self._sentence_tokenized = False
        
#         self._document_view_pointer = 0
        
        self.file_path = file_path 
        # Checks if the file exists
        if os.path.isfile(self.file_path):
            print("The file has been found.")
        else:
            raise IOError("File not found.")
        
        self.raw_docs = []
        self.processed_dict = {}
        
#         # Tokenized sentences
#         self.sent_tokens = []
#         # Flat list of setences
#         self.flat_sent_tokens = []
        
    
    def load_corpus(self):
        with open(self.file_path, "r", encoding="utf8") as corpus:
            data = corpus.readlines()
            for line in data:
                self.raw_docs.append(line)
        
        self._corpus_loaded = True
    
    
    def parse_corpus(self):
        for line in self.raw_docs:
#             parsed_line = json.loads(json_line)
#             self.json_docs.append(parsed_json)
            #print(line)
            pass
        
        self._corpus_parsed = True
    
    
#     def tokenize_sentences(self):

#         for article in self.json_docs:
#             content = article["content"]
#             sent_tokenized = sent_tokenize(content)
#             self.sent_tokens.append(sent_tokenized)
            
#         self.flat_sent_tokens = list(itertools.chain.from_iterable(self.sent_tokens))
        
    
    def print_summary_of_corpus(self):
        if not self._corpus_loaded:
            self.load_corpus()
        
        if not self._corpus_parsed:
            self.parse_corpus()
            
#         if not self._sentence_tokenized:
#             self.tokenize_sentences()
            
        print("Number of training samples: {}.".format(str(len(self.raw_docs))))
        
#         print("Number of documents: {}.".format(str(len(self.json_docs))))
#         print("Number of sentences: {}.".format(str(len(self.flat_sent_tokens))))
    
    
#     def print_document(self, doc_idx):
#         print("-- Printing document with index: {} --".format(doc_idx))
#         print(self.json_docs[doc_idx])

    
#     def get_document(self, doc_idx):
#         return self.json_docs[doc_idx]
    
    
#     def get_next_document(self):        
#         self._document_view_pointer += 1
#         if self._document_view_pointer > len(self.json_docs):
#             return None
#         else:
#             return self.json_docs[(self._document_view_pointer - 1)]
        
    
#     def set_current_document(self, json_doc):
#         self.json_docs[(self._document_view_pointer - 1)] = json_doc
    
    
#     def reset_pointer(self):
#         self._document_view_pointer = 0
        
    
#     def set_pointer(self, pointer):
#         self._document_view_pointer = pointer        


c = Corpus(FILE_NAME)
c.load_corpus()
c.parse_corpus()
c.print_summary_of_corpus()

The file has been found.
2000
2000
Number of training samples: 2000.


In [33]:
import pprint
class X: 
    def __init__(self, val):
        self.val = val
    
    def get_val(self):
        return self.val
    
    def set_val(self, val):
        self.val = val
        
    def __str__(self):
        return "Value: {}".format(str(self.val))

dic = {}

x = X(0)
y = X(2)

dic["0"] = x
dic["1"] = y

for key in dic:
    print("Key: {} Val: {}".format(str(key), str(dic[key])))

a = dic["0"]
b = dic["1"]

a.set_val(100)
b.set_val(1001)

print("printing from dict:")

for key in dic:
    print("Key: {} Val: {}".format(str(key), str(dic[key])))

Key: 0 Val: Value: 0
Key: 1 Val: Value: 2
printing from dict:
Key: 0 Val: Value: 100
Key: 1 Val: Value: 1001


In [12]:



for classifier in ['myclassifier1', 'myclassifier2', 'myclassifier3']: # You may rename the names of the classifiers to something more descriptive
    if classifier == 'myclassifier1':
        print('Training ' + classifier)
        # TODO: extract features for training classifier1
        # TODO: train sentiment classifier1
    elif classifier == 'myclassifier2':
        print('Training ' + classifier)
        # TODO: extract features for training classifier2
        # TODO: train sentiment classifier2
    elif classifier == 'myclassifier3':
        print('Training ' + classifier)
        # TODO: extract features for training classifier3
        # TODO: train sentiment classifier3

    for testset in testsets.testsets:
        # TODO: classify tweets in test set

        predictions = {'163361196206957578': 'neutral', '768006053969268950': 'neutral', '742616104384772304': 'neutral', '102313285628711403': 'neutral', '653274888624828198': 'neutral'} # TODO: Remove this line, 'predictions' should be populated with the outputs of your classifier
        evaluation.evaluate(predictions, testset, classifier)

        evaluation.confusion(predictions, testset, classifier)


Training myclassifier1
twitter-test1.txt (myclassifier1): 0.000
            positive  negative  neutral
positive    0.000     0.000     0.000     
negative    0.000     0.000     0.000     
neutral     0.416     0.158     0.426     

twitter-test2.txt (myclassifier1): 0.000
            positive  negative  neutral
positive    0.000     0.000     0.000     
negative    0.000     0.000     0.000     
neutral     0.530     0.109     0.361     

twitter-test3.txt (myclassifier1): 0.000
            positive  negative  neutral
positive    0.000     0.000     0.000     
negative    0.000     0.000     0.000     
neutral     0.434     0.153     0.413     

Training myclassifier2
twitter-test1.txt (myclassifier2): 0.000
            positive  negative  neutral
positive    0.000     0.000     0.000     
negative    0.000     0.000     0.000     
neutral     0.416     0.158     0.426     

twitter-test2.txt (myclassifier2): 0.000
            positive  negative  neutral
positive    0.000     0.000  